In [1]:
import pickle
import numpy as np

In [2]:
with open ('train_qa.txt', 'rb') as f:
    train_data = pickle.load(f)

In [3]:
with open ('test_qa.txt', 'rb') as f:
    test_data = pickle.load(f)

In [25]:
" ".join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [21]:
 " ".join(train_data[0][1])

'Is Sandra in the hallway ?'

In [23]:
train_data[0][2]

'no'

In [26]:
all_data = test_data + train_data

In [29]:
vocab = set()

for story,question,answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [30]:
vocab.add('no')
vocab.add('yes')

In [31]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [32]:
# Placeholder
vocab_len = len(vocab) + 1

In [34]:
# Longest Story
all_story_lens = [len(data[0]) for data in all_data]

In [36]:
max_story_len = max(all_story_lens)

In [39]:
max_question_len = max([len(data[1]) for data in all_data])

In [40]:
max_question_len

6

In [42]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [44]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [45]:
tokenizer.word_index

{'bathroom': 1,
 'dropped': 2,
 'yes': 3,
 'picked': 4,
 'mary': 5,
 'travelled': 6,
 'milk': 7,
 'football': 8,
 'grabbed': 9,
 'back': 10,
 'daniel': 11,
 'in': 12,
 'left': 13,
 'took': 14,
 'up': 15,
 'to': 16,
 'went': 17,
 'john': 18,
 'hallway': 19,
 'office': 20,
 'is': 21,
 'no': 22,
 'got': 23,
 'apple': 24,
 '.': 25,
 'garden': 26,
 'journeyed': 27,
 'put': 28,
 'the': 29,
 'bedroom': 30,
 'down': 31,
 'discarded': 32,
 'moved': 33,
 'there': 34,
 '?': 35,
 'sandra': 36,
 'kitchen': 37}

In [46]:
train_story_text = []
train_question_text = []
train_answers = []

In [49]:
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [52]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [54]:
# Vectorize the story, question, and answer
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    # Stories = X
    X = []
    
    # Questions Xq
    Xq = []
    
    # Correct Answer (y/n)
    Y = []
    
    for story,query,answer in data:
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        
        y = np.zeros(len(word_index)+1)
        
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))

In [55]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [56]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [57]:
inputs_test

array([[ 0,  0,  0, ..., 29, 30, 25],
       [ 0,  0,  0, ..., 29, 26, 25],
       [ 0,  0,  0, ..., 29, 26, 25],
       ...,
       [ 0,  0,  0, ..., 29, 24, 25],
       [ 0,  0,  0, ..., 29, 26, 25],
       [ 0,  0,  0, ..., 24, 34, 25]])

In [58]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [60]:
tokenizer.word_index['yes']

3

In [61]:
tokenizer.word_index['no']

22

In [63]:
sum(answers_test)

array([  0.,   0.,   0., 497.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
       503.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [65]:
from keras.models import Sequential,Model
from keras.layers.embeddings import Embedding
from keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate,LSTM

In [66]:
# PlaceHolder shape = (max_story_len,batch_size)
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [67]:
# vocab_len
vocab_size = len(vocab) + 1

In [77]:
# Input Encoder M
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.5))

# OUPUT = (samples,story_maxlen, embedding_dim)

In [78]:
# Input Encoder C
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.5))

# OUTPUT = (samples,story_maxlen, embedding_dim)

In [80]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.5))

# OUTPUT = (samples, query_maxlen,embedding_dim)

In [81]:
# ENCODED <----- ENCODER(INPUT)
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [82]:
match = dot([input_encoded_m,question_encoded], axes=(2,2))
match = Activation('softmax')(match)

In [83]:
response = add([match,input_encoded_c])
response= Permute((2, 1))(response)

In [84]:
answer = concatenate([response,question_encoded])

In [85]:
answer

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 6, 220) dtype=float32>

In [87]:
answer = LSTM(32)(answer)

In [88]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer) # y/n

In [89]:
answer = Activation('softmax')(answer)

In [90]:
model = Model([input_sequence,question],answer)

In [92]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

In [93]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 156)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
sequential_8 (Sequential)       multiple             2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_11 (Sequential)      (None, 6, 64)        2432        input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

In [95]:
history = model.fit([inputs_train,queries_train],answers_train,batch_size=32,epochs=100,validation_data=([inputs_test,queries_test],answers_test))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 10000 samples, validate on 1000 samples
Epoch 1/100
10000/10000 [==============================] - 7s 706us/step - loss: 0.9291 - val_loss: 0.6941
Epoch 2/100
10000/10000 [==============================] - 5s 542us/step - loss: 0.7028 - val_loss: 0.6977
Epoch 3/100
10000/10000 [==============================] - 7s 715us/step - loss: 0.6959 - val_loss: 0.6939
Epoch 4/100
10000/10000 [==============================] - 6s 554us/step - loss: 0.6955 - val_loss: 0.6932
Epoch 5/100
10000/10000 [==============================] - 5s 529us/step - loss: 0.6946 - val_loss: 0.6932
Epoch 6/100
10000/10000 [==============================] - 5s 539us/step - loss: 0.6952 - val_loss: 0.6948
Epoch 7/100
10000/10000 [==============================] - 5s 537us/step - loss: 0.6946 - val_loss: 0.6932
Epoch 8/100
10000/10000 [==============================] - 6s 566us/step - l

10000/10000 [==============================] - 5s 545us/step - loss: 0.3085 - val_loss: 0.3447
Epoch 71/100
10000/10000 [==============================] - 5s 523us/step - loss: 0.3084 - val_loss: 0.3477
Epoch 72/100
10000/10000 [==============================] - 5s 494us/step - loss: 0.2981 - val_loss: 0.3462
Epoch 73/100
10000/10000 [==============================] - 5s 518us/step - loss: 0.3022 - val_loss: 0.3420
Epoch 74/100
10000/10000 [==============================] - 5s 511us/step - loss: 0.3011 - val_loss: 0.3503
Epoch 75/100
10000/10000 [==============================] - 5s 522us/step - loss: 0.3020 - val_loss: 0.3479
Epoch 76/100
10000/10000 [==============================] - 5s 538us/step - loss: 0.3003 - val_loss: 0.34160s - lo
Epoch 77/100
10000/10000 [==============================] - 5s 520us/step - loss: 0.2955 - val_loss: 0.3568
Epoch 78/100
10000/10000 [==============================] - 5s 523us/step - loss: 0.3019 - val_loss: 0.3625
Epoch 79/100
10000/10000 [========

In [96]:
model.save('myModel.h5')

In [97]:
model.load_weights('myModel.h5')

In [98]:
pred_results = model.predict(([inputs_test,queries_test]))

In [101]:
pred_results[0]

array([2.7068219e-14, 2.3831916e-14, 2.2445306e-14, 1.7412609e-04,
       2.3667707e-14, 2.3383851e-14, 2.5227948e-14, 2.1530917e-14,
       2.2157000e-14, 2.1932138e-14, 2.3418487e-14, 2.0264421e-14,
       2.2742133e-14, 2.3487063e-14, 2.2494036e-14, 2.6704738e-14,
       2.7183388e-14, 2.5401964e-14, 2.1510270e-14, 2.5087066e-14,
       2.3379659e-14, 2.4692098e-14, 9.9982589e-01, 2.2613669e-14,
       2.3370029e-14, 2.0976386e-14, 2.4991406e-14, 2.4494897e-14,
       2.2416812e-14, 2.3827325e-14, 2.1260480e-14, 2.3496562e-14,
       2.2816953e-14, 2.3988890e-14, 2.2747253e-14, 2.1947369e-14,
       2.5132997e-14, 2.0158639e-14], dtype=float32)

In [102]:
test_data[0][1]

['Is', 'John', 'in', 'the', 'kitchen', '?']

In [103]:
test_data[0][2]

'no'

In [104]:
val_max = np.argmax(pred_results[0])

In [105]:
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key

In [106]:
k

'no'

In [107]:
pred_results[0][val_max]

0.9998259

In [109]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [112]:
# Run your Own Stories
my_story = "John left the kitchen . Sandra dropped the football in the garden . "

In [113]:
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [114]:
my_question = "Is the football in the garden ?"

In [115]:
my_question.split()

['Is', 'the', 'football', 'in', 'the', 'garden', '?']

In [117]:
mydata = [(my_story.split(),my_question.split(), 'yes')]

In [119]:
my_story, my_ques, my_ans = vectorize_stories(mydata)

In [120]:
pred_results = model.predict(([my_story,my_ques]))

In [121]:
val_max = np.argmax(pred_results[0])
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key

In [123]:
k

'yes'